<a href="https://colab.research.google.com/github/nadatanalytics/Modelling_-Project/blob/main/Code_with_sensitivity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Changed code, but droppping X4 (Diversity) and X5 (Elective)

In [ ]:
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils

In [ ]:
from pulp import GLPK
import pandas as pd # Library to handle data
import numpy as np
from statistics import mean

data = pd.read_csv("/content/FinalUnivDataset.csv")
#creating a list to store the optimal decision variables
weights_matrix = np.empty((7,5))
for k in range(5):  
    model = pulp.LpProblem("University Analysis", pulp.LpMaximize) # Create an LP maximization problem
    m1 = pulp.LpVariable("m1", lowBound=0, upBound=None, cat='Continuous') # Create a variable m1>=0
    m2 = pulp.LpVariable("m2", lowBound=0, upBound=None, cat='Continuous') # Create a variable m2>=0
    n1 = pulp.LpVariable("n1", lowBound=0, upBound=None, cat='Continuous') # Create a variable n1>=0
    n2 = pulp.LpVariable("n2", lowBound=0, upBound=None, cat='Continuous') # Create a variable n2>=0
    n3 = pulp.LpVariable("n3", lowBound=0, upBound=None, cat='Continuous') # Create a variable n3>=0
  
   
    model += m1*data["Y1"][k] + m2*data["Y2"][k]  #objective function
    
    model += n1*data["X1"][k] + n2*data["X2"][k] + n3*data["X3"][k] == 1 #first constraint (equality)

    for i in range(5): 
        model += m1*data["Y1"][i] + m2*data["Y2"][i] - n1*data["X1"][i] - n2*data["X2"][i] - n3*data["X3"][i] <= 0
   
    status = model.solve(GLPK(msg=True, options=['--ranges', 'sensitivity.txt'])) 
    print("Model Status:{}".format(pulp.LpStatus[model.status])) 
    


    # Displaying Optimal Decision Variables & Reduced Cost per Variable
    list_weights=[]
    for v in model.variables():
	      print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
    for v in model.variables():  
        list_weights=list_weights+[v.varValue]
    for n in range(5):
        value= list_weights[n]
        weights_matrix[n,k] = value
        
    # Displaying Optimal Value of Objective Function	
    print("Objective=", pulp.value(model.objective))
	
    # Displaying Shadow Price & Slack Value per Constraint
    print ("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
    for name, c in model.constraints.items():
	      print(name, ":", c, "\t", c.pi, "\t\t", c.slack)

Model Status:Optimal
m1 = 0.972222 	Reduced Cost = None
m2 = 0.402778 	Reduced Cost = None
n1 = 0.0 	Reduced Cost = None
n2 = 0.370312 	Reduced Cost = None
n3 = 0.0864583 	Reduced Cost = None
Objective= 0.9999999820000001

Sensitivity Analysis
Constraint		Shadow Price	Slack
_C1 : 28040.5*n1 + 2*n2 + 3*n3 = 1.0 	 None 		 None
_C2 : 0.751*m1 + 0.67*m2 - 28040.5*n1 - 2*n2 - 3*n3 <= -0.0 	 None 		 None
_C3 : 0.955*m1 + 0.771*m2 - 34300.0*n1 - 3*n2 - 5*n3 <= -0.0 	 None 		 None
_C4 : 0.689*m1 + 0.605*m2 - 22153.5*n1 - 2*n2 - 2*n3 <= -0.0 	 None 		 None
_C5 : 0.784*m1 + 0.805*m2 - 19637.5*n1 - 2*n2 - 4*n3 <= -0.0 	 None 		 None
_C6 : 0.627*m1 + 0.395*m2 - 32500.0*n1 - 2*n2 - n3 <= -0.0 	 None 		 None
Model Status:Optimal
m1 = 0.880455 	Reduced Cost = None
m2 = 0.0 	Reduced Cost = None
n1 = 1.78679e-06 	Reduced Cost = None
n2 = 0.239455 	Reduced Cost = None
n3 = 0.0440694 	Reduced Cost = None
Objective= 0.840834525

Sensitivity Analysis
Constraint		Shadow Price	Slack
_C1 : 34300.0*n1 + 3*n2 +

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
efficiency_table =np.empty((5,5))
average_list =[]
for row in range(5):
    row_values=[]
    for column in range(5):
       value = ((weights_matrix[0,column]*data["Y1"][row])+(weights_matrix[1,column]*data["Y2"][row]))/((weights_matrix[2,column]*data["X1"][row])+(weights_matrix[3,column]*data["X2"][row])+(weights_matrix[4,column]*data["X3"][row]))
       efficiency_table[row,column]=value
       row_values=row_values+[value]
    average =mean(row_values)
    average_list =average_list+[average]

In [ ]:
average_list_rounded = [ '%.2f' % elem for elem in average_list ]

In [ ]:
uni_list = ['Warwick','Imperial','Edinburgh','Manchester','Durham']

In [ ]:
for uni in uni_list:
    index = uni_list.index(uni)
    print("Cross-efficiency score for",uni,"=",average_list_rounded[index])

Cross-efficiency score for Warwick = 0.80
Cross-efficiency score for Imperial = 0.65
Cross-efficiency score for Edinburgh = 0.91
Cross-efficiency score for Manchester = 0.81
Cross-efficiency score for Durham = 0.90


In [ ]:
display(efficiency_table)
efficiency_table = efficiency_table.round(decimals=2)
display(efficiency_table)

array([[1.00000108, 1.00000154, 0.77818566, 0.81479146, 0.39925663],
       [0.8028718 , 0.84083545, 0.66113869, 0.6610294 , 0.3046259 ],
       [1.00000115, 1.00000172, 0.99999941, 1.00000419, 0.54944305],
       [1.00000105, 1.00000149, 0.74475624, 1.00000395, 0.3126004 ],
       [0.92938817, 0.95008212, 0.99999919, 0.60378785, 1.0000023 ]])

array([[1.  , 1.  , 0.78, 0.81, 0.4 ],
       [0.8 , 0.84, 0.66, 0.66, 0.3 ],
       [1.  , 1.  , 1.  , 1.  , 0.55],
       [1.  , 1.  , 0.74, 1.  , 0.31],
       [0.93, 0.95, 1.  , 0.6 , 1.  ]])